# Initialization

In [1]:
import matplotlib.pyplot as plt
#%matplotlib inline
from PIL import Image # Install Pillow -> conda install anaconda::pillow or pip install pillow
import os
from skimage.io import  imread, imshow # Install scikit-image -> conda install scikit-image or pip install scikit-image
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import confusion_matrix, balanced_accuracy_score,  mean_squared_error, r2_score, f1_score
from sklearn.model_selection import KFold, train_test_split
import numpy as np

#import torch
import torch.nn as nn
from torch.nn import Dropout
import torch.optim as optim
from torch.functional import F

# DATA LOADING PIPELINE

In [2]:
# Check if the code is running on Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    train_dataset_path = '/content/drive/MyDrive/Project1-AML/data-students/TRAIN'
    test_dataset_path = '/content/drive/MyDrive/Project1-AML/data-students/TEST'
else:
    # Load data from local file
    train_dataset_path = 'data-students/TRAIN'
    test_dataset_path = 'data-students/TEST'

# Now you can use file_path to load your data
#print("File path:", file_path)

#FIXED VARIABLES
IMG_WIDTH = 75 #75
IMG_HEIGHT = 75 #75
BATCH_SIZE = 32

#testing variables
seed = 42

In [3]:
from rembg import remove
import io

def remove_background_pil(image):
    # Convert PIL image to bytes
    with io.BytesIO() as output_buffer:
        image.save(output_buffer, format='PNG')
        image_bytes = output_buffer.getvalue()

    # Use rembg to remove the background
    result = remove(image_bytes)

    # Convert the result binary data back to a PIL image
    result_image = Image.open(io.BytesIO(result))

    # Fill transparent pixels with black
    result_image = fill_transparent_pixels_with_black(result_image)

    # Convert the image to RGB mode if it's not already
    if result_image.mode != 'RGB':
        result_image = result_image.convert('RGB')
        
    return result_image

def fill_transparent_pixels_with_black(image):
    # Convert image to RGBA mode if it's not already
    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # Get the image data as a pixel access object
    pixel_data = image.load()

    # Iterate over each pixel
    width, height = image.size
    for x in range(width):
        for y in range(height):
            # Check if the pixel is transparent
            if pixel_data[x, y][3] == 0:
                # Set the pixel color to black (RGB: 0, 0, 0, Alpha: 255)
                pixel_data[x, y] = (0, 0, 0, 255)

    return image

In [4]:
from torchvision.transforms import v2

# Define your custom transformation function
augmentation = transforms.Compose([v2.RandomPerspective(0.5),v2.RandomAffine(degrees=(-20, 20), translate=(0.1, 0.2), scale=(0.5, 0.9)), v2.ColorJitter(brightness=.1, hue=.05),v2.RandomAdjustSharpness(sharpness_factor=2)])

def removeBackground(image):
    # Apply your custom background removal function
    processed_image = remove_background_pil(image) # remove_background(removal_model, image)
    
    # Apply other transformations if needed
    transform = transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        #transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    return transform(processed_image)

def Augment(image):
    # Apply your custom background removal function
    transform = transforms.Compose([
        augmentation,
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    return transform(image)

def input_transform(image):
    # Apply your custom background removal function
    transform = transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    return transform(image)

normal_transform = transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


In [5]:
#COMBINE DATASET WITH RANDOMLY AUGMENTED DATASETS

torch.cuda.is_available()
True
from torchvision.transforms import v2
import shutil
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

background_removed_path = 'data-students/TRAIN-NOBG'

'''

# Assuming you already have your original dataset and transformed dataset
original_dataset = datasets.ImageFolder(root=train_dataset_path, transform=normal_transform)

# Create a DataLoader
dataloader = DataLoader(original_dataset, batch_size=None, shuffle=False)

transform_to_tensor = transforms.ToTensor()

# Iterate through the DataLoader
for image_path, target in original_dataset.imgs:
    class_name = original_dataset.classes[target]
    class_path = os.path.join(background_removed_path, class_name)
    os.makedirs(class_path, exist_ok=True)
    
    # Load the image
    image = Image.open(image_path)
    
    # Apply the transformation
    transformed_image = transform_to_tensor(removeBackground(image))
    
    # Save the transformed image in the corresponding class folder
    image_filename = os.path.basename(image_path)
    image_save_path = os.path.join(class_path, image_filename)
    transformed_image_pil = transforms.ToPILImage()(transformed_image)
    transformed_image_pil.save(image_save_path)  # Save the transformed image
'''


'\n\n# Assuming you already have your original dataset and transformed dataset\noriginal_dataset = datasets.ImageFolder(root=train_dataset_path, transform=normal_transform)\n\n# Create a DataLoader\ndataloader = DataLoader(original_dataset, batch_size=None, shuffle=False)\n\ntransform_to_tensor = transforms.ToTensor()\n\n# Iterate through the DataLoader\nfor image_path, target in original_dataset.imgs:\n    class_name = original_dataset.classes[target]\n    class_path = os.path.join(background_removed_path, class_name)\n    os.makedirs(class_path, exist_ok=True)\n    \n    # Load the image\n    image = Image.open(image_path)\n    \n    # Apply the transformation\n    transformed_image = transform_to_tensor(removeBackground(image))\n    \n    # Save the transformed image in the corresponding class folder\n    image_filename = os.path.basename(image_path)\n    image_save_path = os.path.join(class_path, image_filename)\n    transformed_image_pil = transforms.ToPILImage()(transformed_image

In [6]:
from torch.utils.data import ConcatDataset
nobg_dataset = datasets.ImageFolder(root=background_removed_path, transform=normal_transform)

# Define how many times you want to enlarge the dataset
enlarge_factor = 5

# Create a list to hold the datasets
combined_datasets = [nobg_dataset]

# Add the transformed dataset to the list multiple times
for _ in range(enlarge_factor):
    transformed_dataset = datasets.ImageFolder(root=background_removed_path, transform=Augment)
    combined_datasets.append(transformed_dataset)

# Concatenate the datasets into a single dataset
enlarged_dataset = ConcatDataset(combined_datasets)


test_set_size = 0.4
#get train & test for K-MEANS

train_val_dataset, test_dataset = train_test_split(enlarged_dataset, test_size = test_set_size, random_state=seed) #random_state = randomizer seed

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(len(enlarged_dataset))
print(len(test_dataset))

3108
1244


# Auxiliary functions


In [7]:
#to print the label (AUXILIAR)
label_str = [
    "12 - Don't Go Left or Right",
    "13 - Don't Go Right",
    "24 - Go Right",
    "37 - Children crossing",
    "38 - Dangerous curve to the right",
    "39 - Dangerous curve to the left",
    "44 - Go left or straight",
    "50 - Fences",
    "6 - Speed limit (70km/h)"
]
label_str_id = [
    "12",
    "13",
    "24",
    "37",
    "38",
    "39",
    "44",
    "50",
    "6"
]

In [8]:
def voting(trained_models,data_loader,type, modelName):
    #type = 1 -> data loader with no labels (test folder)
    #type = 0 -> data loader with labels
    
    save_dir = "Saved_Models"
    
    # Create a directory to save the models if save_dir is provided
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)

    iter = len(trained_models)
    # Save the models
    for i, model in enumerate(trained_models):
        filename = f"fold_{i}_{modelName}.pth"
        if save_dir:
            filename = os.path.join(save_dir, filename)
        torch.save(model, filename)
        

    if type == 0: #data loader with labels
        #pred matrix
        predictions = []
        for i in range(iter):
            filename = f"fold_" + str(i) + "_" + modelName + ".pth"
            torch.save(model, filename)
            model.eval()
            #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            i = 0
            with torch.no_grad():
                    model_predictions = []

                    for inputs, labels in data_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        _, predicted = outputs.max(1)
                        model_predictions.append(predicted)
                    predictions_aux = torch.cat(model_predictions, dim=0).cpu()
                    predictions.append(predictions_aux)

        predictions_matrix = np.vstack(predictions)

        #voting
        num_classes = predictions_matrix.max() + 1

        class_votes = np.zeros((num_classes, predictions_matrix.shape[1])) #(num_classes, num_predictions)

        for col in range(predictions_matrix.shape[1]):
            unique_classes, class_counts = np.unique(predictions_matrix[:, col], return_counts=True)
            class_votes[unique_classes, col] = class_counts

        most_voted_classes = np.argmax(class_votes, axis=0) #pred

        #labels
        full_dataset = []
        for batch in data_loader:
            inputs, labels = batch
            full_dataset.append((inputs, labels))
            y = torch.cat([labels for _, labels in full_dataset], dim=0) #labels

        # evaluation
        # Compute confusion matrix and F1 score
    
        conf_mat = confusion_matrix(y, most_voted_classes)
        f1 = f1_score(y, most_voted_classes, average='weighted')
        bal_acc = balanced_accuracy_score(y, most_voted_classes)
        #precision = precision_score(y, most_voted_classes, average='weighted')
        #recall = recall_score(y, most_voted_classes, average='weighted')

        print('Confusion Matrix:\n', conf_mat)
        print('F1 Score: ', f1)
        print('B_acc: ', bal_acc)
        #print('Precision: ', precision)
        #print('Recall: ', recall)

        return y, most_voted_classes
        
    else: #from test folder - no labels
        predictions = []
        for i in range(iter):
            model = trained_models[i]
            model.eval()
            #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            i = 0
            with torch.no_grad():
                    model_predictions = []

                    for i, inputs in enumerate(data_loader):
                        inputs = inputs.to(device)
                        outputs = model(inputs)
                        predicted = torch.argmax(outputs, dim=1)
                        model_predictions.append(predicted)
                    predictions_aux = torch.cat(model_predictions, dim=0).cpu()
                    predictions.append(predictions_aux)

        predictions_matrix = np.vstack(predictions)

        #voting
        num_classes = predictions_matrix.max() + 1

        class_votes = np.zeros((num_classes, predictions_matrix.shape[1])) #(num_classes, num_predictions)

        for col in range(predictions_matrix.shape[1]):
            unique_classes, class_counts = np.unique(predictions_matrix[:, col], return_counts=True)
            class_votes[unique_classes, col] = class_counts

        most_voted_classes = np.argmax(class_votes, axis=0) #pred
         
        return most_voted_classes

In [9]:
def createCSV(model, test_dataset_loader,type, name):
    import csv

    data = []

    #directory where you want to save the CSV file
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        save_dir = "/content/drive/MyDrive/Project1-AML/Nic/"
    else:
        #save_dir = r"C:\Users\Nicoli Leal\Desktop\MEEC\2 semestre\Aprendizagem Computacional Avançada\Project-1"
        save_dir = "/home/stefanotrenti/AML/project/CSVs"


    #file name
    csv_file = os.path.join(save_dir, name)

    if type == "voting":
      most_voted_classes = voting(model, test_dataset_loader,1,"models_for_voting")
      for i in range(len(most_voted_classes)):
        predicted_class = int(most_voted_classes[i])  # Extract the integer value
        data.append({"ID": i+1, "Class": label_str_id[predicted_class]}) #, "Name": label_str[test_predictions]})

    else:
      for i, images in enumerate(test_dataset_loader):

          images = images.to(device)
          # Forward pass
          outputs = model(images)
          test_predictions = torch.argmax(outputs, dim=1)

          images = images.cpu().numpy()
          predicted_classes = test_predictions.cpu().numpy()

          # Iterate over the batch
          predicted_class = int(predicted_classes[0])  # Extract the integer value
          data.append({"ID": i+1, "Class": label_str_id[predicted_class]}) #, "Name": label_str[test_predictions]})


    # Define the field names
    fields = ["ID", "Class"]#, "Name"]

    # Write data to CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fields)

        # Write the header
        writer.writeheader()

        # Write the data rows
        for row in data:
            writer.writerow(row)

    print("CSV file created successfully.")
    return

In [10]:
import os,sys

def saveResults(model, directory, name, modelName, loss, optim):
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    # Join the directory path with the file name
    results_path = os.path.join(directory, name)
    
    # Check if the file exists
    file_exists = os.path.isfile(results_path)
    
    # Open the file in append mode or write mode depending on whether the file exists
    with open(results_path, 'a' if file_exists else 'w') as f:
        # Redirect stdout to the file
        sys.stdout = f
        print(" ---------------------------------------------------------------------------------------------------------------------- ")
        print(" ----------------------------- Model parameters ------------------------------")
        print("model name:" + modelName)
        print('learning rate  = 0.001')
        print('epochs = 50')
        print('folds = 5')
        print('batch size = 64')
        print('Test set size  = ',test_set_size)
        print("enlarge_factor =", enlarge_factor)
        print('loss = '+ loss)
        print('optimizer = '+ optim)
        print(" ------------------------------- Evaluation  ---------------------------------")
        evaluate_network(model, test_loader)
        print(" ---------------------------------------------------------------------------------------------------------------------- ")
 
    # Reset stdout to its default value (console)
    sys.stdout = sys.__stdout__


# EVALUATION FUNCTION

In [11]:
def evaluate_network(model,dataset_loader, to_device=True):
    # X given input data
    # y corresponding target labels
    full_dataset = []
    for batch in dataset_loader:
        # Assuming each batch is a tuple (inputs, labels)
        inputs, labels = batch
        full_dataset.append((inputs, labels))

        # Concatenate all data points into a single tensor
        X = torch.cat([inputs for inputs, _ in full_dataset], dim=0)
        y = torch.cat([labels for _, labels in full_dataset], dim=0)


    # Set the model to evaluation mode
    model.eval()
    if to_device:
      # Assuming you're using GPU (if available)
        #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        X = X.to(device)
        y = y.to(device)
        #model = model.to(device)

    # Run the model on the test data
    with torch.no_grad():
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)

    # Convert tensors to numpy arrays
    if to_device:
        predicted = predicted.to("cpu")

    predicted_np = predicted.cpu().numpy()
    test_target_np = y.cpu().numpy()


    # Compute confusion matrix and F1 score
    conf_mat = confusion_matrix(test_target_np, predicted_np)
    f1 = f1_score(test_target_np, predicted_np, average='weighted')
    bal_acc = balanced_accuracy_score(y.cpu(), predicted_np)

    #print('Confusion Matrix:\n', conf_mat)
    print('F1 Score: ', f1)
    print('B_acc: ', bal_acc)

    return conf_mat, f1, bal_acc

# TRAIN Function

In [12]:
def trainCNN(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100. * correct / total

        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    print('Finished Training')
    return model, val_acc, val_loss

In [13]:
def Train_K_FOLDS(k_model, epochs, lr, folds, batch_size, loss_type, optim_type):
    
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    
    print("Optimizer type:", optim_type)
    print("Loss type:", loss_type)

    models = []
    ordered_models = []
    index=0
    i = 0
    best_accuracy = 0
    val_values = []

    for fold, (train_index, val_index) in enumerate(kf.split(train_val_dataset)):
        
        i= i+1
        
        model = k_model()

        #Creating DataLoaders for training and validation
        train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_index)
        #
        train_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=val_sampler)

        #CHOOSE LOSS
        if loss_type == 'MSELoss' or loss_type == 'BCELoss':
            criterion = nn.MSELoss()
            
        elif loss_type == 'cross_entropy':
            criterion = nn.CrossEntropyLoss()
        elif loss_type == 'BCELoss':
            criterion = nn.BCELoss()
        else:
            raise ValueError(f"Unsupported loss type: {loss_type}")
            
        #CHOOSE OPTIMIZER
        if optim_type == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=lr)
        elif optim_type == 'SGD':
            optimizer = optim.SGD(model.parameters(), lr=lr)
        elif optim_type == 'RMSprop':
            optimizer = optim.RMSprop(model.parameters(), lr=lr)
        else:
            raise ValueError(f"Unsupported optimizer type: {optim_type}")
            
        # Train the model

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if loss_type == 'MSELoss' or loss_type == 'BCELoss':
            trained_model, val_acc, val_loss = trainCNN_mse(model, train_loader, val_loader, criterion, optimizer, epochs, device)
        else:
            trained_model, val_acc, val_loss = trainCNN(model, train_loader, val_loader, criterion, optimizer, epochs, device)
        
        models.append(trained_model)
        #conf_matrix, F_score, bal_acc = evaluate_network(trained_model, val_loader)
        
        val_values.append(val_acc)
        
        if(val_acc > best_accuracy):
            best_accuracy = val_acc
            index = i
            best_model = model
        

        print("Best model was with fold number ", index , "\n")
          
    # Create a list of tuples with (value, index)
    indexed_values = [(value, index) for index, value in enumerate(val_values)]
    # Sort the list of tuples based on the values (in descending order)
    sorted_values = sorted(indexed_values, key=lambda x: x[0], reverse=True)
    # Extract the sorted indexes
    sorted_indexes = [index for value, index in sorted_values]
    
    first_value = models[sorted_indexes[0]]
    second_value = models[sorted_indexes[1]]
    third = models[sorted_indexes[2]]
    fourth = models[sorted_indexes[3]]
    fifth = models[sorted_indexes[4]]
    ordered_models.append(first_value)  
    ordered_models.append(second_value)
    ordered_models.append(third)
    ordered_models.append(fourth)
    ordered_models.append(fifth)    
        
    return best_model, ordered_models


In [14]:
'''
def Train_K_FOLDS(k_model, epochs, lr, folds, batch_size):
    
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    models = []
    ordered_models = []
    index=0
    i = 0
    best_accuracy = 0
    val_values = []

    for fold, (train_index, val_index) in enumerate(kf.split(train_val_dataset)):
        
        i= i+1
        
        model = k_model()

        #Creating DataLoaders for training and validation
        train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_index)
        #
        train_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=val_sampler)

        # Define your loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Train the model

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        trained_model, val_acc, val_loss = trainCNN(model, train_loader, val_loader, criterion, optimizer, epochs, device)
        models.append(trained_model)
        #conf_matrix, F_score, bal_acc = evaluate_network(trained_model, val_loader)
        
        val_values.append(val_acc)
        
        if(val_acc > best_accuracy):
            best_accuracy = val_acc
            index = i
            best_model = model
        

        print("Best model was with fold number ", index , "\n")
          
    # Create a list of tuples with (value, index)
    indexed_values = [(value, index) for index, value in enumerate(val_values)]
    # Sort the list of tuples based on the values (in descending order)
    sorted_values = sorted(indexed_values, key=lambda x: x[0], reverse=True)
    # Extract the sorted indexes
    sorted_indexes = [index for value, index in sorted_values]
    
    first_value = models[sorted_indexes[0]]
    second_value = models[sorted_indexes[1]]
    third = models[sorted_indexes[2]]
    fourth = models[sorted_indexes[3]]
    fifth = models[sorted_indexes[4]]
    ordered_models.append(first_value)  
    ordered_models.append(second_value)
    ordered_models.append(third)
    ordered_models.append(fourth)
    ordered_models.append(fifth)    
        
    return best_model, ordered_models
    '''

'\ndef Train_K_FOLDS(k_model, epochs, lr, folds, batch_size):\n    \n    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)\n\n    models = []\n    ordered_models = []\n    index=0\n    i = 0\n    best_accuracy = 0\n    val_values = []\n\n    for fold, (train_index, val_index) in enumerate(kf.split(train_val_dataset)):\n        \n        i= i+1\n        \n        model = k_model()\n\n        #Creating DataLoaders for training and validation\n        train_sampler = torch.utils.data.SubsetRandomSampler(train_index)\n        val_sampler = torch.utils.data.SubsetRandomSampler(val_index)\n        #\n        train_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=train_sampler)\n        val_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=val_sampler)\n\n        # Define your loss function and optimizer\n        criterion = nn.CrossEntropyLoss()\n        optimizer = optim.Adam(model.parameters(), lr=lr)\n\n        # Train the model\n\n      

# CNN

In [15]:
class DV_CNN_v3(nn.Module):
    def __init__(self, input_channels=3, num_classes=9):
        super(DV_CNN_v3, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(16)

        self.fc1 = nn.Linear(9 * 9 * 16, 120)
        self.fc2 = nn.Linear(120, num_classes)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(F.relu(self.bn3(self.conv3(x))), 2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(F.relu(self.bn5(self.conv5(x))), 2)
        x = F.relu(self.bn6(self.conv6(x)))

        x = x.view(-1, self.fc1.in_features)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def print_architecture(self):
        print(self)
        
    def get_architecture_string(self):
        return str(self)
    
    def get_architecture_name(self):
        return str('DV_CNN_v3')

# TRAINING THE CNN

In [16]:
save_dir = '/home/stefanotrenti/AML/project/AOL_test'
name = 'AOL_function_tests.txt'

optimizers = ['adam','SGD','RMSprop']
losses = ['cross_entropy','MSELoss','BCELoss']
epochs = 50

DV_CNN_v3_relu, DV_CNN_v3_relu_models = Train_K_FOLDS(DV_CNN_v3, 50, 0.001, 5, 64, 'cross_entropy', 'SGD')

print('Done training!')

modelName_relu = DV_CNN_v3_relu.get_architecture_name() + ' 50 epochs'

saveResults(DV_CNN_v3_relu,save_dir,name,modelName_relu, 'cross_entropy', 'SGD')

models_for_voting_ALL = []

models_for_voting_ALL.extend(DV_CNN_v3_relu_models)

Optimizer type: SGD
Loss type: cross_entropy


/home/stefanotrenti/miniconda3/envs/AML-env/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025831440/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/50], Train Loss: 2.1887, Train Acc: 15.69%, Val Loss: 2.1884, Val Acc: 14.75%
Epoch [2/50], Train Loss: 2.1312, Train Acc: 26.09%, Val Loss: 2.1196, Val Acc: 26.27%
Epoch [3/50], Train Loss: 2.0894, Train Acc: 29.04%, Val Loss: 2.0732, Val Acc: 27.35%
Epoch [4/50], Train Loss: 2.0555, Train Acc: 29.51%, Val Loss: 2.0354, Val Acc: 29.22%
Epoch [5/50], Train Loss: 2.0224, Train Acc: 31.19%, Val Loss: 2.0040, Val Acc: 29.22%
Epoch [6/50], Train Loss: 1.9892, Train Acc: 31.19%, Val Loss: 1.9705, Val Acc: 29.22%
Epoch [7/50], Train Loss: 1.9574, Train Acc: 31.99%, Val Loss: 1.9434, Val Acc: 30.29%
Epoch [8/50], Train Loss: 1.9234, Train Acc: 32.86%, Val Loss: 1.9079, Val Acc: 29.76%
Epoch [9/50], Train Loss: 1.8830, Train Acc: 34.27%, Val Loss: 1.8679, Val Acc: 32.17%
Epoch [10/50], Train Loss: 1.8599, Train Acc: 34.88%, Val Loss: 1.8308, Val Acc: 33.24%
Epoch [11/50], Train Loss: 1.8044, Train Acc: 37.69%, Val Loss: 1.7935, Val Acc: 34.58%
Epoch [12/50], Train Loss: 1.7788, Train 

EVALUATE NETWORK ON GENERATED TEST SET

In [17]:
evaluate_network(DV_CNN_v3_relu, test_loader)

(array([[222,   0,   0,   2,   0,   0,   0,   1,   8],
        [ 55,   1,  13,   0,   0,   0,   0,   0,   6],
        [  0,   0, 256,   0,   0,   0,   0,   0,   0],
        [  4,   0,   0, 123,   0,   0,   0,  20,   0],
        [  5,   0,   0,  32,   0,   0,   1,  27,   0],
        [  9,   0,   4,  31,   0,   0,   2,  48,   0],
        [  1,   0,   1,  24,   0,   0,   5,  36,   0],
        [  8,   1,   0,  58,   0,   0,   0,  72,   0],
        [  1,   0,   0,   0,   0,   0,   0,   0, 167]]),
 0.6079835110153993,
 0.4877242025568614)

## LOAD TEST FOLDER DATA

In [18]:
import re
from torch.utils.data import Dataset

class TestDataset(Dataset):
    def get_int(self, text):
        return [int(c) if c.isdigit() else c for c in re.split('(\d+)', text)]

    def __init__(self, images_folder, transform=None):
        self.images_folder = images_folder
        self.image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]
        self.image_files.sort(key=self.get_int)
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_folder, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image
    
    
'''   
    
#run if we want to use new test folder every time we use csv

transform_test = transforms.Compose([removeBackground,input_transform])

inference_dataset = TestDataset(images_folder=test_dataset_path, transform=transform_test)

test_dataset_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

'''

#IF YOU WANT TO TAKE IMAGES FROM THE TEST FLDER WITH NO BG

test_nobg_path = 'data-students/TEST-NOBG'

inference_dataset = TestDataset(images_folder=test_nobg_path, transform=input_transform)

test_dataset_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)



# CREATE AND SAVE CSV FILE WITH PREDICTION RESULTS

In [19]:
createCSV(DV_CNN_v3_relu, test_dataset_loader,"","DV_CNN_v3_relu.csv")

#createCSV(models_for_voting_best, test_dataset_loader,"voting","predictions_CNN_voting_best.csv")
createCSV(models_for_voting_ALL, test_dataset_loader,"voting","predictions_CNN_voting_ALL.csv")

F1 Score:  0.6079835110153993
B_acc:  0.4877242025568614
CSV file created successfully.
CSV file created successfully.


## END